# Weibull RNN Model

# 1 - Information

In [ ]:
# Author: Pierre Oreistein

# 2 - Packages

In [ ]:
%reload_kedro

In [ ]:
# Math Packages
import numpy as np

# Data Handling Packages
import pandas as pd
import json

# TensorFlow
import tensorflow as tf

# Keras
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, LSTM
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.activations import relu
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, History, TerminateOnNaN
from tensorflow.keras.optimizers import Nadam

# Attention
#from attention import attention_3d_block

# Prevent unecessary warnings
from warnings import filterwarnings
filterwarnings("ignore", ".*`should_run_async`.*")

# 3 - RNN Model: WTTE

## 3.1 - Check for GPU

In [ ]:
print("Is a GPU available?", bool(len(tf.config.list_physical_devices('GPU'))))

## 3.2 - Custom Loss and Activation

In [ ]:
def clipping_activation(bbeta: tf.Tensor, name: str=None) -> tf.Tensor:
    """Relu activation with clipping to avoid exploding parameters

       Args:
        bbeta is a (samples, 2) tensor containing the parameters b and beta of the Weibull distribution
    """

    # Apply a relu activation
    b = K.clip(relu(bbeta[:, 0]), 10e-5, 10e5)
    beta = K.clip(relu(bbeta[:, 1]), 10e-5, 10e5)

    # Reshape the two parameters to vectors
    b = K.reshape(b, (K.shape(b)[0], 1))
    beta = K.reshape(beta, (K.shape(beta)[0], 1))

    return K.concatenate((b, beta), axis=1)

In [ ]:
def custom_loss(y_true: tf.Tensor, bbeta_pred: tf.Tensor, name: str=None) -> tf.Tensor:
    """ Customn loss for the maximising the Weibull log-likelihood

        Args:
            y_true: tensor with last dimension having length 2
                with y_true[:,0] = time to event,
                     y_true[:,1] = indicator of not censored

            y_pred: tensor with last dimension having length 2 
                with y_pred[:,0] = alpha,
                     y_pred[:,1] = beta

        Returns:
            A positive `Tensor` of same shape as input
    """

    # Extract b, beta, y, u
    b = bbeta_pred[:, 0]
    beta = bbeta_pred[:, 1]
    y = y_true[:, 0]
    u = y_true[:, 1]

    # Compute the log-likelihood for u == 1 and u == 0
    hazard_u_1 = b * K.exp(beta * K.log(y)) - (beta - 1) * K.log(y) - K.log(beta) - K.log(b) # uncensored data
    hazard_u_0 = - K.log(1 - K.exp(-b * K.exp(beta * K.log(y)))) # censored data
    
    loglikelihoods = u * hazard_u_1 + (1 - u) * hazard_u_0
    loss = K.mean(loglikelihoods)

    return loss

## 3.3 - Model

In [ ]:
def create_rnn_model(nb_features: int = 15) -> tf.keras.Model:
    """Create RNN model"""
    # Redifine epsilon
    K.set_epsilon(1e-20)

    # Callbacks
    nanterminator = TerminateOnNaN()
    callbacks = [nanterminator]

    # Metrics
    metrics = []

    # Start building our model
    i = Input(batch_shape=(None, 1, nb_features))
    x = LSTM(10, return_sequences=True, unroll=True)(i) # returns a sequence of vectors of dimension 32
    x = attention_3d_block(x)
    x = Dense(2, activation=clipping_activation)(x)
    model = Model(inputs=[i], outputs=[x])

    # Compile the model
    model.compile(optimizer=Adam(), loss=custom_loss, metrics=metrics)

    # Display the model summary
    model.summary()

    return model

In [ ]:
def train_rnn_model(model: tf.keras.Model, train_df: pd.DataFrame) -> tf.keras.Model:
    """Train the baseline model."""
    # Extract X and Y
    X_train_df = train_df
    y_train_df = X_train_df.pop('RUL')
    
    # Train the baseline model
    model.fit(
        X_train_df,
        y_train_df,
        epochs=DEEP_LEARNING_PARAMS_DCT["epoch"],
        batch_size=DEEP_LEARNING_PARAMS_DCT["batch-size"]
    )
    
    return model

In [ ]:
# Load Deep Learning params
DEEP_LEARNING_PARAMS_DCT = context.params["deep-learning-parameters"]
DEEP_LEARNING_PARAMS_DCT = {k: v for d in DEEP_LEARNING_PARAMS_DCT for k, v in d.items()}

# Load the training dataset
train_df = catalog.load("train_preprocessed_df")

# Create the baseline model
nb_features = int(DEEP_LEARNING_PARAMS_DCT["nb_features"])
model = create_rnn_model(nb_features=nb_features)

# Train the baseline model
model = train_rnn_model(model=model, train_df=train_df)